#  PikaPikaGenerator - Training Model 
# 
**Progetto:** Generative Synthesis of Pokémon Sprites from Textual Descriptions  
 **Corso:** Deep Learning - Politecnico di Bari  
 **Studente:** Pasquale Alessandro Denora  
 **Professore:** Vito Walter Anelli 

#  Import e Setup Iniziale
 
 Il file inizia importando tutte le librerie necessarie per il training del modello:
 - **torch & torch.optim**: Framework PyTorch e ottimizzatori
 - **torch.cuda.amp**: Automatic Mixed Precision per training accelerato
 - **torch.utils.tensorboard**: Logging e visualizzazione metriche
 - **tqdm**: Progress bar per monitoraggio training
 - **wandb**: Weights & Biases per experiment tracking avanzato
 - **PIL**: Per salvare immagini generate durante il training

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from tqdm import tqdm
from pathlib import Path
import json
import logging
from typing import Dict, Optional, Tuple
from datetime import datetime
import wandb
from PIL import Image

from src.models.architecture import create_model, Discriminator
from src.data.preprocessing import create_dataloaders

logger = logging.getLogger(__name__)

#  Classe Trainer - Inizializzazione
 
 La classe `Trainer` (righe 27-627) gestisce tutto il processo di training. Nel costruttore vengono inizializzati:
 
 **Setup del dispositivo** (riga 31): Seleziona automaticamente GPU se disponibile, altrimenti CPU
 
 **Parametri config utilizzati**:
 - `config['project']['device']`: dispositivo preferito
 - `config['loss']['adversarial_weight']`: peso per training adversarial
 
 **Componenti inizializzati**:
 - Modello principale (generatore)
 - Discriminatore (se training adversarial abilitato)
 - Ottimizzatori e scheduler
 - Loss functions
 - Sistema di logging


In [ ]:
class Trainer:
    
    def __init__(self, config: Dict):
        self.config = config
        self.device = torch.device(config['project']['device'] if torch.cuda.is_available() else 'cpu')
        
        # Crea cartelle necessarie
        self.setup_directories()
        
        # Inizializza il modello
        self.model = create_model(config).to(self.device)
        
        # Inizializza il discriminatore per l'addestramento avversariale
        self.use_adversarial = config['loss']['adversarial_weight'] > 0
        if self.use_adversarial:
            self.discriminator = Discriminator(config).to(self.device)
        
        # Inizializza l'ottimizzatore
        self.setup_optimizers()
        
        # Inizializza la loss functions
        self.setup_losses()
        
        
        self.scaler = GradScaler() if self.device.type == 'cuda' else None
        
        # Logging
        self.setup_logging()
        
        # Traccio il miglior modello
        self.best_val_loss = float('inf')
        self.patience_counter = 0

#  Setup Directory e Gestione File
 
 Il metodo `setup_directories()` (righe 60-67) crea automaticamente tutte le cartelle necessarie per il training:
 
 **Directory create**:
 - `checkpoints_dir`: Per salvare i modelli durante il training
 - `samples_dir`: Per salvare esempi di immagini generate
 - `logs_dir`: Per i log di TensorBoard
 
 **Parametri config utilizzati**:
 - `config['paths']['checkpoints_dir']`: path checkpoints (es: "data/models/checkpoints_hq_v2")
 - `config['paths']['samples_dir']`: path samples (es: "outputs/samples_hq_v2")  
 - `config['paths']['logs_dir']`: path logs (es: "logs")


In [ ]:
def setup_directories(self):
    """Creo le cartelle necessarie"""
    self.checkpoint_dir = Path(self.config['paths']['checkpoints_dir'])
    self.samples_dir = Path(self.config['paths']['samples_dir'])
    self.logs_dir = Path(self.config['paths']['logs_dir'])
    
    for dir_path in [self.checkpoint_dir, self.samples_dir, self.logs_dir]:
        dir_path.mkdir(parents=True, exist_ok=True)

#  Setup Ottimizzatori e Scheduler
 
 Il metodo `setup_optimizers()` (righe 69-94) configura gli ottimizzatori per generatore e discriminatore:
 
 **Ottimizzatore Generatore** (righe 72-76):
 - Usa Adam optimizer su tutti i parametri del modello
 - Learning rate e beta parameters da config
 
 **Ottimizzatore Discriminatore** (righe 79-84):
 - Solo se training adversarial abilitato
 - Learning rate 2x rispetto al generatore (comune pratica GAN)
 
 **Scheduler Learning Rate** (righe 87-94):
 - ReduceLROnPlateau: riduce LR quando validation loss smette di migliorare
 - Pazienza di 5 epoche, riduzione factor 0.5
 
 **Parametri config utilizzati**:
 - `config['training']['learning_rate']`: learning rate base (0.0001)
 - `config['training']['beta1']`, `config['training']['beta2']`: parametri Adam (0.0, 0.99)


In [ ]:
def setup_optimizers(self):
    """Setto gli ottimizzatori e gli scheduler"""
    # Genera gli ottimizzatori
    self.optimizer_g = optim.Adam(
        self.model.parameters(),
        lr=self.config['training']['learning_rate'],
        betas=(self.config['training']['beta1'], self.config['training']['beta2'])
    )
    
    # Ottimizzatore per il discriminatore se usato
    if self.use_adversarial:
        self.optimizer_d = optim.Adam(
            self.discriminator.parameters(),
            lr=self.config['training']['learning_rate'] * 2, 
            betas=(self.config['training']['beta1'], self.config['training']['beta2'])
        )
    
    # Schedulatore learning rate
    self.scheduler_g = optim.lr_scheduler.ReduceLROnPlateau(
        self.optimizer_g, mode='min', patience=5, factor=0.5
    )
    
    if self.use_adversarial:
        self.scheduler_d = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer_d, mode='min', patience=5, factor=0.5
        )

#  Setup Loss Functions
 
 Il metodo `setup_losses()` (righe 96-113) inizializza tutte le loss function utilizzate:
 
 **L1 Loss** (riga 99): Loss di ricostruzione pixel-wise, preferita rispetto a L2 per immagini meno sfocate
 
 **Perceptual Loss LPIPS** (righe 102-109): 
 - Misura similarità semantica usando deep features
 - Più importante della semplice similarità pixel-wise
 - Gestisce ImportError se libreria LPIPS non disponibile
 
 **Adversarial Loss** (righe 112-113):
 - BCEWithLogitsLoss per classificazione real/fake
 - Solo se training adversarial abilitato
 
 **Parametri config utilizzati**:
 - `config['loss']['perceptual_weight']`: peso loss perceptuale (2.5)
 - `config['loss']['adversarial_weight']`: peso loss adversarial (0.5)

In [ ]:
def setup_losses(self):
    """Setto la  loss functions"""
    # Ricostruzione L1 loss
    self.l1_loss = nn.L1Loss()
    
    # Loss perceptuale (LPIPS)
    if self.config['loss']['perceptual_weight'] > 0:
        try:
            import lpips
            self.perceptual_loss = lpips.LPIPS(net='alex').to(self.device)
            logger.info("LPIPS perceptual loss initialized")
        except ImportError:
            logger.warning("LPIPS not available, disabling perceptual loss")
            self.config['loss']['perceptual_weight'] = 0
    
    # Loss avversariale
    if self.use_adversarial:
        self.adversarial_loss = nn.BCEWithLogitsLoss()


#  Setup Logging - TensorBoard e Weights & Biases
 
 Il metodo `setup_logging()` (righe 115-133) configura i sistemi di logging per monitorare il training:
 
 **TensorBoard** (righe 118-119):
 - Crea SummaryWriter con timestamp per log unici
 - Salva nella directory logs con nome run_YYYYMMDD_HHMMSS
 
 **Weights & Biases** (righe 122-133):
 - Sistema avanzato per experiment tracking e comparison
 - Include model watching per tracking automatico gradients
 - Gestisce errori di connessione gracefully
 
 **Parametri config utilizzati**:
 - `config['logging']['use_wandb']`: abilita W&B (false nel tuo config)
 - `config['logging']['project_name']`: nome progetto W&B ("pikapika-high-quality-v2")



In [ ]:
def setup_logging(self):
    """Setto il logging e TensorBoard"""
    # TensorBoard
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    self.writer = SummaryWriter(self.logs_dir / f'run_{timestamp}')
    
    # Weights & Biases
    if self.config['logging'].get('use_wandb', False):
        try:
            wandb.init(
                project=self.config['logging']['project_name'],
                config=self.config,
                name=f"run_{timestamp}"
            )
            wandb.watch(self.model)
            logger.info("W&B logging initialized")
        except Exception as e:
            logger.warning(f"Failed to initialize W&B: {e}")
            self.config['logging']['use_wandb'] = False


#  Calcolo Metriche di Valutazione
 
 Il metodo `compute_metrics()` (righe 135-257) calcola metriche comprehensive per valutare la qualità delle immagini generate:
 
 **Metriche implementate**:
 1. **SSIM** (righe 149-173): Structural Similarity Index, misura similarità strutturale
 2. **PSNR** (righe 175-188): Peak Signal-to-Noise Ratio, qualità di ricostruzione  
 3. **LPIPS** (righe 190-202): Learned Perceptual Image Patch Similarity
 4. **FID approssimato** (righe 205-225): Fréchet Inception Distance semplificato
 5. **IS approssimato** (righe 227-237): Inception Score semplificato
 
 **Robustezza**: Ogni metrica ha error handling completo per evitare crash durante training


In [ ]:
def compute_metrics(self, real_images: torch.Tensor, fake_images: torch.Tensor) -> Dict[str, float]:
    """Compute evaluation metrics with comprehensive error handling"""
    try:
        metrics = {}
        
        # Ensure images are in valid range
        real_images = torch.clamp(real_images, -1, 1)
        fake_images = torch.clamp(fake_images, -1, 1)
        
        # Convert to numpy for some metrics
        real_np = real_images.detach().cpu().numpy()
        fake_np = fake_images.detach().cpu().numpy()
        
        # SSIM
        try:
            from skimage.metrics import structural_similarity as ssim
            ssim_scores = []
            for i in range(min(real_np.shape[0], 8)):  # Limita a 8 immagini per efficienza
                # Converti da [-1,1] a [0,1] e trasponi HWC
                real_img = np.transpose((real_np[i] + 1) / 2, (1, 2, 0))
                fake_img = np.transpose((fake_np[i] + 1) / 2, (1, 2, 0))
                
                # Converti a scala di grigi
                real_gray = np.mean(real_img, axis=2)
                fake_gray = np.mean(fake_img, axis=2)
                
                # Clippa i valori per evitare errori
                real_gray = np.clip(real_gray, 0, 1)
                fake_gray = np.clip(fake_gray, 0, 1)
                
                score = ssim(real_gray, fake_gray, data_range=1.0)
                if not np.isnan(score) and not np.isinf(score):
                    ssim_scores.append(score)
            
            metrics['ssim'] = float(np.mean(ssim_scores)) if ssim_scores else 0.5
            
        except Exception as e:
            logger.debug(f"SSIM calculation failed: {e}")
            metrics['ssim'] = 0.5

#  Training Step - Cuore del Training
 
 Il metodo `train_step()` (righe 259-359) implementa un singolo step di training:
 
 **Flusso di training**:
 1. **Move batch to device** (righe 263-268): Sposta dati su GPU/CPU
 2. **Train Discriminatore** (righe 271-301): Se adversarial training abilitato
    - Genera fake images con generatore
    - Classifica real vs fake images  
    - Calcola adversarial loss per discriminatore
 3. **Train Generatore** (righe 304-332): Training principale
    - Calcola reconstruction loss (L1)
    - Calcola perceptual loss (LPIPS) se abilitata
    - Calcola adversarial loss per fooling discriminatore
 4. **Backpropagation** (righe 335-340): Con gradient clipping e AMP support
 
 **Parametri config utilizzati**:
 - `config['loss']['reconstruction_weight']`: peso L1 loss (15.0)
 - `config['loss']['perceptual_weight']`: peso LPIPS loss (2.5)
 - `config['loss']['adversarial_weight']`: peso adversarial loss (0.5)

In [ ]:
def train_step(self, batch: Dict) -> Dict[str, float]:
    """Single training step"""
    try:
        # Move batch to device
        images = batch['image'].to(self.device)
        input_ids = batch['input_ids'].to(self.device)
        attention_mask = batch['attention_mask'].to(self.device)
        
        batch_size = images.shape[0]
        losses = {}
        
        # Train discriminator (if using adversarial training)
        if self.use_adversarial:
            self.optimizer_d.zero_grad()
            
            with autocast(enabled=self.scaler is not None):
                # Genera immagini fake
                with torch.no_grad():
                    outputs = self.model(input_ids, attention_mask)
                    fake_images = outputs['generated_image']
                
                # Predizioni del discriminatore
                real_pred = self.discriminator(images)
                fake_pred = self.discriminator(fake_images.detach())
                
                # Labels
                real_labels = torch.ones_like(real_pred)
                fake_labels = torch.zeros_like(fake_pred)
                
                # Loss di discriminatore
                d_loss_real = self.adversarial_loss(real_pred, real_labels)
                d_loss_fake = self.adversarial_loss(fake_pred, fake_labels)
                d_loss = (d_loss_real + d_loss_fake) / 2

#  Validazione del Modello
 
 Il metodo `validate()` (righe 361-442) esegue la validazione del modello su validation set:
 
 **Processo di validazione**:
 1. **Model eval mode** (riga 363): Disabilita dropout e batch norm training
 2. **No gradient computation** (riga 368): Risparmia memoria e accelera validazione  
 3. **Loop sui batch** (righe 369-381): Calcola loss e metriche su ogni batch
 4. **Calcolo metriche** (righe 383-387): Solo sui primi 3 batch per efficienza
 5. **Aggregazione risultati** (righe 398-427): Media delle loss e metriche
 
 **Robustezza**:
 - Error handling per ogni batch per evitare crash
 - Valori di default se computation fallisce
 - Validation di tutti i valori per evitare NaN/Inf
 
 **Output**: Dizionario con val_loss e tutte le metriche (SSIM, PSNR, FID, IS, LPIPS)


In [ ]:
def validate(self, val_loader) -> Dict[str, float]:
        """Validation loop with robust error handling"""
        self.model.eval()
        val_losses = []
        val_metrics = []
        
        try:
            with torch.no_grad():
                for batch_idx, batch in enumerate(tqdm(val_loader, desc="Validating")):
                    try:
                        images = batch['image'].to(self.device)
                        input_ids = batch['input_ids'].to(self.device)
                        attention_mask = batch['attention_mask'].to(self.device)
                        
                        # Genera immagini
                        outputs = self.model(input_ids, attention_mask)
                        generated_images = outputs['generated_image']
                        
                        # Calcola le loss
                        recon_loss = self.l1_loss(generated_images, images)
                        val_losses.append(float(recon_loss.item()))
                        
                        # Calcola le metriche
                        if batch_idx < 3:  # Calcola le metriche solo per i primi 3 batch
                            metrics = self.compute_metrics(generated_images, images)
                            if metrics is not None:
                                val_metrics.append(metrics)
                        
                    except Exception as e:
                        logger.warning(f"Error in validation batch {batch_idx}: {e}")
                        continue
            
            # Gestisce i casi in cui non sono state calcolate metriche valide
            if not val_losses:
                logger.warning("No valid losses computed during validation")
                return {'val_loss': 1.0, 'ssim': 0.5, 'psnr': 20.0, 'fid': 50.0, 'is_score': 2.0, 'lpips': 0.5}
            
            # Calcola la loss media
            avg_loss = float(np.mean(val_losses))
            
            # Calcola le metriche medie
            if val_metrics:
                # Filtra i None values
                valid_metrics = [m for m in val_metrics if m is not None and isinstance(m, dict)]
                
                if valid_metrics:
                    avg_metrics = {}
                    for key in valid_metrics[0].keys():
                        values = []
                        for m in valid_metrics:
                            if key in m and np.isfinite(m[key]):
                                values.append(m[key])
                        
                        if values:
                            avg_metrics[key] = float(np.mean(values))
                        else:
                            # Valori predefiniti per le metriche mancanti
                            defaults = {'ssim': 0.5, 'psnr': 20.0, 'fid': 50.0, 'is_score': 2.0, 'lpips': 0.5}
                            avg_metrics[key] = defaults.get(key, 0.0)
                else:
                    # Se non ci sono metriche valide, usa valori predefiniti
                    avg_metrics = {'ssim': 0.5, 'psnr': 20.0, 'fid': 50.0, 'is_score': 2.0, 'lpips': 0.5}
            else:
                # Nessuna metrica calcolata, usa valori predefiniti
                avg_metrics = {'ssim': 0.5, 'psnr': 20.0, 'fid': 50.0, 'is_score': 2.0, 'lpips': 0.5}
            
            result = {'val_loss': avg_loss, **avg_metrics}
            
            # Convalida che tutti i valori siano finiti
            for key, value in result.items():
                if not np.isfinite(value):
                    logger.warning(f"Invalid validation metric {key}: {value}")
                    defaults = {'val_loss': 1.0, 'ssim': 0.5, 'psnr': 20.0, 'fid': 50.0, 'is_score': 2.0, 'lpips': 0.5}
                    result[key] = defaults.get(key, 0.0)
            
            self.model.train()
            return result
            
        except Exception as e:
            logger.error(f"Error in validation: {e}")
            self.model.train()
            return {'val_loss': 1.0, 'ssim': 0.5, 'psnr': 20.0, 'fid': 50.0, 'is_score': 2.0, 'lpips': 0.5}

#  Salvataggio Checkpoint
 
 Il metodo `save_checkpoint()` (righe 444-470) salva lo stato completo del training:
 
 **Contenuto checkpoint** (righe 446-457):
 - Numero epoca corrente
 - State dict del modello (pesi e parametri)
 - State dict dell'ottimizzatore (momentum, learning rate)
 - Metriche di validazione dell'epoca
 - Configurazione completa del progetto
 - Se adversarial: anche discriminatore e suo ottimizzatore
 
 **Due tipi di salvataggio**:
 1. **Checkpoint regolare**: Salvato ogni N epoche per recovery
 2. **Best model**: Salvato solo quando validation loss migliora
 
 **Parametri config utilizzati**:
 - `config['training']['save_every']`: frequenza salvataggio (20 epoche)

In [ ]:
def save_checkpoint(self, epoch: int, val_metrics: Dict, is_best: bool = False):
    """Save model checkpoint"""
    try:
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_g_state_dict': self.optimizer_g.state_dict(),
            'val_metrics': val_metrics,
            'config': self.config
        }
        
        if self.use_adversarial:
            checkpoint['discriminator_state_dict'] = self.discriminator.state_dict()
            checkpoint['optimizer_d_state_dict'] = self.optimizer_d.state_dict()
        
        # Save regular checkpoint
        checkpoint_path = self.checkpoint_dir / f'checkpoint_epoch_{epoch}.pt'
        torch.save(checkpoint, checkpoint_path)
        
        # Save best model
        if is_best:
            best_path = self.checkpoint_dir / 'best_model.pt'
            torch.save(checkpoint, best_path)
            logger.info(f"Saved best model with val_loss: {val_metrics['val_loss']:.4f}")
            
    except Exception as e:
        logger.error(f"Error saving checkpoint: {e}")

#  Generazione Campioni per Monitoraggio
 
 Il metodo `generate_samples()` (righe 472-515) genera immagini di esempio durante il training:
 
 **Scopo**: Monitorare visivamente il progresso del modello attraverso le epoche
 
 **Sample texts predefiniti** (righe 478-487): 8 descrizioni diverse che coprono vari tipi di Pokemon:
 - Pokemon elettrico (Pikachu-like)
 - Pokemon acquatico (Blastoise-like)  
 - Pokemon drago (Charizard-like)
 - Pokemon pianta (Venusaur-like)
 - E altri tipi per diversità
 
 **Processo** (righe 491-510):
 1. Modello in eval mode
 2. Genera immagine per ogni descrizione  
 3. Salva ogni immagine singolarmente
 4. Ritorna modello a train mode
 
 **Parametri config utilizzati**:
 - `config['training']['sample_every']`: frequenza generazione campioni (5 epoche)


In [ ]:
def generate_samples(self, epoch: int, num_samples: int = 8):
        """Generate and save sample images"""
        try:
            self.model.eval()
            
            # Descrizione dei campioni
            sample_texts = [
                "A small yellow electric mouse Pokemon with red cheeks and a lightning bolt tail",
                "A large blue turtle Pokemon with water cannons on its shell",
                "An orange dragon Pokemon that breathes fire and has wings",
                "A green plant Pokemon with a large flower on its back",
                "A purple ghost Pokemon with a mischievous smile",
                "A pink fairy Pokemon with ribbons and bows",
                "A steel bird Pokemon with sharp metallic feathers",
                "A dark wolf Pokemon with red eyes and sharp claws"
            ][:num_samples]
            
            generated_images = []
            
            with torch.no_grad():
                for i, text in enumerate(sample_texts):
                    try:
                        # Genera l'immagine
                        image = self.model.generate(text, device=self.device)
                        generated_images.append(image)
                        
                        # Salva la singola immagine
                        img = Image.fromarray(image)
                        img.save(self.samples_dir / f'epoch_{epoch}_sample_{i}.png')
                        
                    except Exception as e:
                        logger.warning(f"Error generating sample {i}: {e}")
                        continue
            
            if generated_images:
                logger.info(f"Generated {len(generated_images)} samples for epoch {epoch}")
            
            self.model.train()
            return self.samples_dir / f'epoch_{epoch}_samples'
            
        except Exception as e:
            logger.error(f"Error generating samples: {e}")
            self.model.train()
            return None

#  Loop Principale di Training
 
 Il metodo `train()` (righe 517-628) orchestra l'intero processo di training:
 
 **Struttura del loop principale**:
 1. **Loop epoche** (righe 526-527): Itera attraverso tutte le epoche
 2. **Training loop** (righe 530-557): Training su tutti i batch
 3. **Progress monitoring** (righe 560-569): Aggiorna progress bar e log metriche
 4. **Validation** (righe 572-584): Ogni N epoche secondo config
 5. **Early stopping** (righe 604-606): Se validation non migliora per N epoche
 6. **Sample generation** (righe 609-614): Genera esempi per monitoraggio visivo
 
 **Gestione errori robusta**:
 - Continue se singolo batch fallisce
 - KeyboardInterrupt per stop manuale
 - Finally block per cleanup (chiude writer, W&B)
 
 **Parametri config utilizzati**:
 - `config['training']['validate_every']`: frequenza validazione (5 epoche)
 - `config['training']['patience']`: pazienza per early stopping (80 epoche)
 - `config['logging']['log_every']`: frequenza logging (10 step)

In [ ]:
def train(self, train_loader, val_loader, num_epochs: int):
        """Main training loop"""
        logger.info(f"Starting training for {num_epochs} epochs")
        logger.info(f"Device: {self.device}")
        logger.info(f"Model parameters: {sum(p.numel() for p in self.model.parameters()):,}")
        
        global_step = 0
        
        try:
            for epoch in range(1, num_epochs + 1):
                logger.info(f"\nEpoch {epoch}/{num_epochs}")
                
                # Training loop
                epoch_losses = []
                progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch}")
                
                for batch_idx, batch in enumerate(progress_bar):
                    try:
                        # Training step
                        losses = self.train_step(batch)
                        epoch_losses.append(losses)
                        
                        # Aggiorna la progress bar
                        progress_bar.set_postfix({k: f"{v:.4f}" for k, v in losses.items()})
                        
                        # Logging
                        if global_step % self.config['logging']['log_every'] == 0:
                            for key, value in losses.items():
                                self.writer.add_scalar(f'train/{key}', value, global_step)
                            
                            if self.config['logging'].get('use_wandb', False):
                                try:
                                    wandb.log({f'train/{k}': v for k, v in losses.items()}, step=global_step)
                                except Exception as e:
                                    logger.debug(f"W&B logging failed: {e}")
                        
                        global_step += 1
                        
                    except Exception as e:
                        logger.warning(f"Error in training batch {batch_idx}: {e}")
                        continue
                
                # Media delle perdite dell'epoca
                if epoch_losses:
                    avg_losses = {}
                    for key in epoch_losses[0].keys():
                        values = [loss.get(key, 0.0) for loss in epoch_losses if isinstance(loss, dict)]
                        avg_losses[key] = float(np.mean(values)) if values else 0.0
                    
                    logger.info(f"Epoch {epoch} - Average losses: {avg_losses}")
                else:
                    logger.warning(f"No valid losses for epoch {epoch}")
                    continue
                
                # Validation
                if epoch % self.config['training']['validate_every'] == 0:
                    val_metrics = self.validate(val_loader)
                    logger.info(f"Validation metrics: {val_metrics}")
                    
                    # Validation metrics logging
                    for key, value in val_metrics.items():
                        self.writer.add_scalar(f'val/{key}', value, epoch)
                    
                    if self.config['logging'].get('use_wandb', False):
                        try:
                            wandb.log({f'val/{k}': v for k, v in val_metrics.items()}, step=global_step)
                        except Exception as e:
                            logger.debug(f"W&B validation logging failed: {e}")
                    
                    # Schedulazione learning rate
                    self.scheduler_g.step(val_metrics['val_loss'])
                    if self.use_adversarial:
                        self.scheduler_d.step(val_metrics['val_loss'])
                    
                    # Controlla se il modello è il migliore
                    is_best = val_metrics['val_loss'] < self.best_val_loss
                    if is_best:
                        self.best_val_loss = val_metrics['val_loss']
                        self.patience_counter = 0
                    else:
                        self.patience_counter += 1
                    
                    # Salva il checkpoint
                    if epoch % self.config['training']['save_every'] == 0 or is_best:
                        self.save_checkpoint(epoch, val_metrics, is_best)
                    
                    # Stop anticipato
                    if self.patience_counter >= self.config['training']['patience']:
                        logger.info(f"Early stopping triggered after {epoch} epochs")
                        break
                
                # Genera esempi
                if epoch % self.config['training']['sample_every'] == 0:
                    sample_path = self.generate_samples(epoch)
                    if sample_path:
                        logger.info(f"Generated samples saved to {sample_path}")
            
            logger.info("Training completed successfully!")
            
        except KeyboardInterrupt:
            logger.info("Training interrupted by user")
        except Exception as e:
            logger.error(f"Training failed: {e}")
            raise
        finally:
            self.writer.close()
            if self.config['logging'].get('use_wandb', False):
                try:
                    wandb.finish()
                except:
                    pass

#  Funzione Train Model - Entry Point
 
 La funzione `train_model()` (righe 630-652) è il punto di ingresso principale per il training:
 
 **Setup iniziale** (righe 632-633):
 - Imposta seed per riproducibilità usando config seed (42)
 - Garantisce risultati consistenti tra run diversi
 
 **Creazione componenti** (righe 635-642):
 - Crea dataloaders usando la funzione dal preprocessing
 - Usa il tokenizer specificato in config per consistency
 - Istanzia la classe Trainer con tutta la configurazione
 
 **Avvio training** (righe 645-651):
 - Chiama il metodo train con train/val loader
 - Usa numero epoche da config
 - Ritorna trainer instance per analisi post-training
 
 **Parametri config utilizzati**:
 - `config['project']['seed']`: seed riproducibilità (42)
 - `config['model']['encoder']['model_name']`: tokenizer name ("prajjwal1/bert-mini")
 - `config['training']['num_epochs']`: numero epoche totali (300)


In [ ]:
def train_model(config: Dict):
    # Setta seeds casuali
    torch.manual_seed(config['project']['seed'])
    np.random.seed(config['project']['seed'])
    
    # Crea dataloaders
    dataloaders = create_dataloaders(
        config,
        tokenizer_name=config['model']['encoder']['model_name']
    )
    
    # Crea trainer
    trainer = Trainer(config)
    
    # Addestra il modello
    trainer.train(
        train_loader=dataloaders['train'],
        val_loader=dataloaders['val'],
        num_epochs=config['training']['num_epochs']
    )
    
    return trainer

#  Script di Test del Training
 
 Il blocco finale (righe 654-660) permette di testare il training quando si esegue il file direttamente:
 
 **Test eseguiti**:
 1. **Caricamento config**: Legge configurazione da configs/config.yaml
 2. **Avvio training completo**: Esegue train_model() con config reale
 3. **Monitoraggio**: Tutti i log, checkpoints, samples vengono generati
 
 Questo è utile per:
 - Testare l'intero pipeline di training
 - Debug di configurazioni diverse
 - Avvio training standalone senza main.py


In [ ]:
if __name__ == "__main__":
    import yaml
    
    with open('configs/config.yaml', 'r') as f:
        config = yaml.safe_load(f)
    
    trainer = train_model(config)

#  Riepilogo Parametri Config Utilizzati
 
 Il file `trainer.py` utilizza quasi tutti i parametri dal file `config.yaml`:
 
 **Sezione `training`**:
 - `learning_rate`: Learning rate ottimizzatori (0.0001)
 - `beta1`, `beta2`: Parametri Adam optimizer (0.0, 0.99)
 - `num_epochs`: Numero epoche totali (300)
 - `batch_size`: Dimensione batch (2) - usato nei dataloaders
 - `validate_every`: Frequenza validazione (5 epoche)
 - `save_every`: Frequenza salvataggio checkpoint (20 epoche)
 - `sample_every`: Frequenza generazione campioni (5 epoche)
 - `patience`: Pazienza early stopping (80 epoche)
 
 **Sezione `loss`**:
 - `reconstruction_weight`: Peso L1 loss (15.0)
 - `perceptual_weight`: Peso LPIPS loss (2.5)
 - `adversarial_weight`: Peso adversarial loss (0.5)

 
 **Sezione `logging`**:
 - `log_every`: Frequenza logging TensorBoard (10 steps)
 - `use_wandb`: Abilita Weights & Biases (false)
 - `project_name`: Nome progetto W&B ("pikapika-high-quality-v2")
 
 **Sezione `paths`**:
 - `checkpoints_dir`: Directory checkpoint ("data/models/checkpoints_hq_v2")
 - `samples_dir`: Directory samples ("outputs/samples_hq_v2")
 - `logs_dir`: Directory logs TensorBoard ("logs")
 
 **Sezione `project`**:
 - `device`: Dispositivo preferito ("cpu")
 - `seed`: Seed riproducibilità (42)

#  Conclusioni - Sistema di Training PikaPikaGenerator
 
 Il sistema di training implementato in `trainer.py` è completo e production-ready:
 
  **Metriche di Valutazione**:
 - **SSIM**: Similarità strutturale immagini
 - **PSNR**: Qualità ricostruzione pixel-wise  
 - **LPIPS**: Similarità percettuale deep learning
 - **FID/IS**: Qualità distribuzione immagini generate
  **Workflow Completo**:
 1. **Setup**: Directories, model, optimizers, losses
 2. **Training Loop**: Batch processing con progress monitoring
 3. **Validation**: Calcolo metriche comprehensive ogni N epoche  
 4. **Checkpointing**: Salvataggio automatico migliori modelli
 5. **Sample Generation**: Monitoring visivo progresso
 6. **Early Stopping**: Terminazione intelligente training